In [26]:
# environment set up
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import ast
from scipy import stats as st

In [27]:
# working folder = "/Users/Srikar/Desktop/Velexi/spectra-ml/data"

spectrum_len = 500 # automate this
parent_dir = os.environ['PWD']
stddata_path = os.path.join(os.environ['DATA_DIR'], "StdData-" + str(spectrum_len))
os.chdir(os.path.join(parent_dir, "lab-notebook", "smunukutla"))

In [28]:
# data extraction

data = pd.read_csv("data.csv", sep=",")
record_nums = data.iloc[0, :].tolist()
spectrum_names = data.iloc[1, :].tolist()
y = data.iloc[2, :].astype(int).tolist()
y = np.reshape(y, (len(y), 1))
num_samples = len(y)

In [29]:
spectra = np.zeros((num_samples,spectrum_len))

# wavelengths = np.zeros((1,spectrum_len))
# y = np.zeros((num_samples, 1))

In [30]:
for i in range(len(record_nums)):
    data = pd.read_csv(os.path.join(stddata_path,"{}.csv".format(record_nums[i])))
#     if i == 0:
#         wavelengths[i,:] = data.iloc[:, 0].to_numpy()
    spectra[i,:] = data.iloc[:, 1].to_numpy()

In [31]:
y_cat = to_categorical(y)

In [34]:
fi = open("indices.txt", "r")
num_runs = int(fi.readline())
num_minerals = int(fi.readline())

stats = []

for i in range(num_runs):
    train_set_indices = ast.literal_eval(fi.readline())
    test_set_indices = ast.literal_eval(fi.readline())
    dev_set_indices = ast.literal_eval(fi.readline())
    
    for j in train_set_indices:
        j = int(j)
    for k in test_set_indices:
        k = int(k)
    for m in dev_set_indices:
        m = int(m)
    
    train_set = spectra[train_set_indices, :]
#     train_labels = y[train_set_indices, :]
    dev_set = spectra[dev_set_indices, :]
#     dev_labels = y[dev_set_indices, :]
    test_set = spectra[test_set_indices, :]
#     test_labels = y[test_set_indices, :]

#     train_labels = train_labels.flatten()
#     dev_labels = dev_labels.flatten()
#     test_labels = test_labels.flatten()

    train_set = np.reshape(train_set, (train_set.shape[0], spectrum_len, 1))
    dev_set = np.reshape(dev_set, (dev_set.shape[0], spectrum_len, 1))
    test_set = np.reshape(test_set, (test_set.shape[0], spectrum_len, 1))

#     train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
#     dev_labels = np.reshape(dev_labels, (dev_labels.shape[0], 1))
#     test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

#     train_labels = to_categorical(train_labels)
#     dev_labels = to_categorical(dev_labels)
#     test_labels = to_categorical(test_labels)

    train_labels = y_cat[train_set_indices, :]
    dev_labels = y_cat[dev_set_indices, :]
    test_labels = y_cat[test_set_indices, :]
    
#     print(train_labels)
    
    model = Sequential() # tf upgrading to 2.0, after that we need to specify the dtype/construct all layers at once
    # model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
    model.add(Conv1D(64, 25, activation='relu', input_shape=(train_set.shape[1], 1))) # optional: , dtype=tf.dtypes.float64
    model.add(Conv1D(64, 25, activation='relu'))
    model.add(MaxPooling1D(4)) # 108 by 64 so far
    model.add(Conv1D(100, 25, activation='relu'))
    model.add(Conv1D(100, 25, activation='relu'))
    model.add(MaxPooling1D(4))
    # model.add(Dropout(0.5))
    # model.add(GlobalAveragePooling1D())
    model.add(Flatten())
    model.add(Dense(num_minerals, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    BATCH_SIZE = 32
    EPOCHS = 25

    model.fit(train_set, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(dev_set, dev_labels))
    
    my_list = model.evaluate(test_set, test_labels, verbose=0)

#     print(tf.keras.losses.categorical_crossentropy())
    
#     print(model.predict(test_set))
    
#     print(model.layers[0].get_weights())

#     from tensorflow.keras import backend as K

#     # with a Sequential model
#     get_3rd_layer_output = K.function([model.layers[0].input],
#                                       [model.layers[7].output])
#     layer_output = get_3rd_layer_output(dev_set)
#     print(layer_output)
    
#     print(dev_labels[13])
#     print(dev_set[13])
    
    stats.append(my_list[1])

# print("1D CNN:", stats)
print("1D CNN Results:", st.describe(stats))

Train on 99 samples, validate on 33 samples
Epoch 1/25
99/99 [==============================] - 4s 39ms/sample - loss: 2.2777 - acc: 0.0707 - val_loss: 2.2301 - val_acc: 0.1515
Epoch 2/25
99/99 [==============================] - 1s 8ms/sample - loss: 2.2093 - acc: 0.1717 - val_loss: 2.2158 - val_acc: 0.2727
Epoch 3/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1738 - acc: 0.2424 - val_loss: 2.2630 - val_acc: 0.2727
Epoch 4/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1503 - acc: 0.2929 - val_loss: 2.1228 - val_acc: 0.2727
Epoch 5/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.0744 - acc: 0.2929 - val_loss: 2.0495 - val_acc: 0.2727
Epoch 6/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.9770 - acc: 0.3030 - val_loss: 2.2119 - val_acc: 0.2727
Epoch 7/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.9994 - acc: 0.3030 - val_loss: 1.8953 - val_acc: 0.3636
Epoch 8/25
99/99 [=============

99/99 [==============================] - 1s 9ms/sample - loss: 1.8681 - acc: 0.2828 - val_loss: 1.7573 - val_acc: 0.3333
Epoch 12/25
99/99 [==============================] - 1s 9ms/sample - loss: 1.7550 - acc: 0.3131 - val_loss: 1.8854 - val_acc: 0.4242
Epoch 13/25
99/99 [==============================] - 1s 10ms/sample - loss: 1.6367 - acc: 0.4040 - val_loss: 1.6961 - val_acc: 0.3939
Epoch 14/25
99/99 [==============================] - 1s 11ms/sample - loss: 1.7657 - acc: 0.3737 - val_loss: 4.6545 - val_acc: 0.1212
Epoch 15/25
99/99 [==============================] - 1s 11ms/sample - loss: 2.6307 - acc: 0.2424 - val_loss: 2.0304 - val_acc: 0.3939
Epoch 16/25
99/99 [==============================] - 1s 12ms/sample - loss: 2.1974 - acc: 0.2828 - val_loss: 1.8336 - val_acc: 0.3636
Epoch 17/25
99/99 [==============================] - 1s 10ms/sample - loss: 1.8407 - acc: 0.3535 - val_loss: 1.8554 - val_acc: 0.3636
Epoch 18/25
99/99 [==============================] - 1s 9ms/sample - loss: 1

Epoch 22/25
99/99 [==============================] - 1s 9ms/sample - loss: 1.0494 - acc: 0.5657 - val_loss: 2.0741 - val_acc: 0.4545
Epoch 23/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.0985 - acc: 0.6061 - val_loss: 1.5750 - val_acc: 0.4848
Epoch 24/25
99/99 [==============================] - 1s 8ms/sample - loss: 0.9951 - acc: 0.6465 - val_loss: 1.2378 - val_acc: 0.5758
Epoch 25/25
99/99 [==============================] - 1s 8ms/sample - loss: 0.9865 - acc: 0.6667 - val_loss: 1.3333 - val_acc: 0.6364
Train on 99 samples, validate on 33 samples
Epoch 1/25
99/99 [==============================] - 5s 46ms/sample - loss: 2.3090 - acc: 0.0707 - val_loss: 2.2422 - val_acc: 0.1515
Epoch 2/25
99/99 [==============================] - 1s 12ms/sample - loss: 2.2281 - acc: 0.1010 - val_loss: 2.2736 - val_acc: 0.1515
Epoch 3/25
99/99 [==============================] - 1s 11ms/sample - loss: 2.2509 - acc: 0.1515 - val_loss: 2.2124 - val_acc: 0.2727
Epoch 4/25
99/99 [=======

99/99 [==============================] - 1s 8ms/sample - loss: 2.2638 - acc: 0.1515 - val_loss: 2.0752 - val_acc: 0.3030
Epoch 8/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1198 - acc: 0.3030 - val_loss: 2.1821 - val_acc: 0.4545
Epoch 9/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1813 - acc: 0.3838 - val_loss: 2.0846 - val_acc: 0.3333
Epoch 10/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.0902 - acc: 0.3030 - val_loss: 1.9597 - val_acc: 0.2727
Epoch 11/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.0234 - acc: 0.3030 - val_loss: 1.8142 - val_acc: 0.3636
Epoch 12/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.8756 - acc: 0.3535 - val_loss: 3.6045 - val_acc: 0.0909
Epoch 13/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.4679 - acc: 0.2727 - val_loss: 1.8380 - val_acc: 0.3939
Epoch 14/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.0304 -

99/99 [==============================] - 1s 7ms/sample - loss: 1.6871 - acc: 0.4343 - val_loss: 1.6024 - val_acc: 0.4242
Epoch 19/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.5646 - acc: 0.4141 - val_loss: 1.4886 - val_acc: 0.3939
Epoch 20/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.4745 - acc: 0.4242 - val_loss: 1.5647 - val_acc: 0.4242
Epoch 21/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.4993 - acc: 0.4545 - val_loss: 2.0555 - val_acc: 0.3636
Epoch 22/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.7064 - acc: 0.3737 - val_loss: 1.8332 - val_acc: 0.4242
Epoch 23/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.6889 - acc: 0.3939 - val_loss: 1.6588 - val_acc: 0.3636
Epoch 24/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.5214 - acc: 0.4242 - val_loss: 1.6327 - val_acc: 0.4242
Epoch 25/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.5307

Epoch 4/25
99/99 [==============================] - 1s 8ms/sample - loss: 2.2223 - acc: 0.1515 - val_loss: 2.2120 - val_acc: 0.1515
Epoch 5/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.3134 - acc: 0.1515 - val_loss: 2.1837 - val_acc: 0.1515
Epoch 6/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.2350 - acc: 0.1717 - val_loss: 2.2839 - val_acc: 0.1818
Epoch 7/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.2859 - acc: 0.1717 - val_loss: 2.2782 - val_acc: 0.0909
Epoch 8/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.2729 - acc: 0.1111 - val_loss: 2.2448 - val_acc: 0.0909
Epoch 9/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.2348 - acc: 0.1010 - val_loss: 2.1811 - val_acc: 0.0909
Epoch 10/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1523 - acc: 0.2020 - val_loss: 2.0941 - val_acc: 0.1818
Epoch 11/25
99/99 [==============================] - 1s 7ms/sample - loss: 

99/99 [==============================] - 1s 7ms/sample - loss: 1.4214 - acc: 0.4949 - val_loss: 0.9755 - val_acc: 0.6364
Epoch 16/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.2570 - acc: 0.5152 - val_loss: 1.2436 - val_acc: 0.4242
Epoch 17/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.4615 - acc: 0.4343 - val_loss: 1.2546 - val_acc: 0.5152
Epoch 18/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.2233 - acc: 0.5152 - val_loss: 0.9781 - val_acc: 0.5758
Epoch 19/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.2139 - acc: 0.5859 - val_loss: 0.9833 - val_acc: 0.6061
Epoch 20/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.1010 - acc: 0.5354 - val_loss: 1.0305 - val_acc: 0.5758
Epoch 21/25
99/99 [==============================] - 1s 8ms/sample - loss: 1.0920 - acc: 0.5859 - val_loss: 0.9413 - val_acc: 0.6364
Epoch 22/25
99/99 [==============================] - 1s 8ms/sample - loss: 0.9960

Epoch 1/25
99/99 [==============================] - 4s 44ms/sample - loss: 2.2680 - acc: 0.0808 - val_loss: 2.3306 - val_acc: 0.1515
Epoch 2/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.2627 - acc: 0.1515 - val_loss: 2.2112 - val_acc: 0.1515
Epoch 3/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.2076 - acc: 0.1212 - val_loss: 2.1967 - val_acc: 0.1818
Epoch 4/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1912 - acc: 0.1212 - val_loss: 2.1940 - val_acc: 0.2121
Epoch 5/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1555 - acc: 0.1212 - val_loss: 2.1142 - val_acc: 0.2424
Epoch 6/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.1415 - acc: 0.1818 - val_loss: 2.0851 - val_acc: 0.2424
Epoch 7/25
99/99 [==============================] - 1s 7ms/sample - loss: 2.0706 - acc: 0.2020 - val_loss: 2.1130 - val_acc: 0.2727
Epoch 8/25
99/99 [==============================] - 1s 7ms/sample - loss: 2

99/99 [==============================] - 1s 7ms/sample - loss: 1.3900 - acc: 0.5051 - val_loss: 1.8373 - val_acc: 0.4242
Epoch 13/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.6766 - acc: 0.3939 - val_loss: 1.5367 - val_acc: 0.4848
Epoch 14/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.5482 - acc: 0.4242 - val_loss: 1.6842 - val_acc: 0.4545
Epoch 15/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.6847 - acc: 0.4040 - val_loss: 1.6783 - val_acc: 0.3333
Epoch 16/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.6466 - acc: 0.4747 - val_loss: 1.4996 - val_acc: 0.4848
Epoch 17/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.4136 - acc: 0.4848 - val_loss: 1.4013 - val_acc: 0.4848
Epoch 18/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.2287 - acc: 0.4949 - val_loss: 1.3531 - val_acc: 0.4242
Epoch 19/25
99/99 [==============================] - 1s 7ms/sample - loss: 1.2997

In [6]:
# # random.seed()
# model = Sequential()
# # model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
# model.add(Conv1D(64, 25, activation='relu', input_shape=(train_set.shape[1], 1)))
# model.add(Conv1D(64, 25, activation='relu'))
# model.add(MaxPooling1D(4)) # 108 by 64 so far
# model.add(Conv1D(100, 25, activation='relu'))
# model.add(Conv1D(100, 25, activation='relu'))
# model.add(MaxPooling1D(4))
# # model.add(Dropout(0.5))
# # model.add(GlobalAveragePooling1D())
# model.add(Flatten())
# model.add(Dense(3, activation='softmax'))
# print(model.summary())